In [2]:
!nvidia-smi

Sat Jun  6 18:57:30 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.64.00    Driver Version: 440.64.00    CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla M40           On   | 00000000:04:00.0 Off |                    0 |
| N/A   29C    P8    14W / 250W |      0MiB / 11448MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  Tesla M40           On   | 00000000:82:00.0 Off |                    0 |
| N/A   29C    P8    16W / 250W |      0MiB / 11448MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
        

In [8]:
%load_ext tensorboard

# load tensorflow dependencies
import tensorflow as tf
from classification_models.tfkeras import Classifiers
from tensorflow.keras.utils import to_categorical
from tensorflow_addons.metrics import CohenKappa
from tensorflow.keras import layers as KL
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau
import tensorflow.keras.backend as K
import efficientnet.tfkeras as efn

# 16 bit precision computing
from tensorflow.keras.mixed_precision import experimental as mixed_precision
print(tf.__version__)
print(tf.config.experimental.list_physical_devices())


from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import os
from pathlib import Path
from pprint import pprint
import pandas as pd
from glob import glob
import skimage.io
import sys

from tensorflow.keras.utils import Progbar


from tqdm import tqdm

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import cohen_kappa_score, make_scorer
import time


sys.path.insert(0,'..')
from model.layers import GeneralizedMeanPooling2D

# custom packages
from preprocessing.utils.data_loader import PandasDataLoader
from preprocessing.generators import TiffGenerator, TiffFromCoords
from utils.utils import set_gpu_memory, seed_all

from model.network import Network

# Augmentation packages

# custom stain augmentation
from preprocessing.augmentations import StainAugment

import albumentations as A
from albumentations import (
    Flip, ShiftScaleRotate, RandomRotate90,
    ShiftScaleRotate, Blur, OpticalDistortion, RandomBrightnessContrast, 
    OneOf, Compose, RandomScale, ElasticTransform, GaussNoise, GaussianBlur,
    RandomBrightness, RandomContrast
)

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard
2.2.0
[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:XLA_CPU:0', device_type='XLA_CPU'), PhysicalDevice(name='/physical_device:XLA_GPU:0', device_type='XLA_GPU'), PhysicalDevice(name='/physical_device:XLA_GPU:1', device_type='XLA_GPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]


In [3]:
seed_all(20)
DATA_DIR = Path('../data/')

# control for the gpu memory, and number of used gpu's
set_gpu_memory(device_type='GPU')
# see the utils functions which seeds are set
# tensorflow ops still have to be seeded manually...
seed_all()

policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_policy(policy)

print('Compute dtype: %s' % policy.compute_dtype)
print('Variable dtype: %s' % policy.variable_dtype)

2 Physical GPUs, 1 Logical GPUs
Compute dtype: float16
Variable dtype: float32


In [4]:
# ------------------
# Directories
# ------------------
DATA_DIR = '../data'  # General path to the data dir
IMG_DIR = '../data/train_images'  # Path to the TILED images
TRAIN_MASKS_DIR = '../data/masks'  # Path to the masks

# ------------------
# Data constants
# ------------------
NFOLDS = 4  # number of folds to use for training/validation (cross validation)
SEED=5  # the seed TODO: REPLACE THIS WITH A FUNCTION THAT SEEDS EVERYTHING WITH SEED
TRAIN_FOLD=0  # select the first fold for training/validation

# ------------------
# Network parameters
# ------------------
# some of these parameters might disappear in a future release
SZ = 256  # width and heigth of the image
NUM_CLASSES = 6 
BATCH_SIZE = 5
NUM_EPOCHS = 15  
NUM_TILES = 16 # this number cannot be changed freely in the png tile generator
LEARNING_RATE = 1e-3
TILE_LEVEL = 2

tile_params = {'N': NUM_TILES, 'sz': SZ}

In [5]:
# an example: loading the skip dataframe and listing the possible reasons
skip_df = pd.read_csv(Path(DATA_DIR) / Path('PANDA_Suspicious_Slides_15_05_2020.csv'))
print("possible faulty slide reasons", skip_df['reason'].unique())

fold_df = PandasDataLoader(images_csv_path=Path(DATA_DIR) / Path('train.csv'),
                           skip_csv=Path(DATA_DIR) / Path('PANDA_Suspicious_Slides_15_05_2020.csv'), 
                           skip_list=[])

# we create a possible stratification here, the options are by isup grade, or further distilled by isup grade and data provider
# stratified_isup_sample or stratified_isup_dp_sample, we use the former.

fold_df = fold_df.stratified_isup_sample(NFOLDS, SEED)

# we can create training/validation splits from the fold column
train_df = fold_df[fold_df['split'] != TRAIN_FOLD]
valid_df = fold_df[fold_df['split'] == TRAIN_FOLD]

display(train_df)
display(valid_df)
print(len(train_df))

possible faulty slide reasons ['marks' 'No Mask' 'Background only'
 'No cancerous tissue but ISUP Grade > 0' 'tiss' 'blank']
********************
The training dataframe shape before filtering:(10616, 4)
The skip dataframe has shape: (675, 2), with reasons ['marks', 'No Mask', 'Background only', 'No cancerous tissue but ISUP Grade > 0', 'tiss', 'blank']
Filtering based on the following columns: ['marks', 'No Mask', 'Background only', 'No cancerous tissue but ISUP Grade > 0', 'tiss', 'blank']
number of duplicates in the skip df: (13, 2)
Training dataframe after filtering: (9954, 4)
Number of rows removed by filter: 662
********************


,image_id,data_provider,isup_grade,gleason_score,split
0,0005f7aaab2800f6170c399693a96917,karolinska,0,0+0,1
1,000920ad0b612851f8e01bcc880d9b3d,karolinska,0,0+0,3
2,0018ae58b01bdadc8e347995b69f99aa,radboud,4,4+4,2
3,001c62abd11fa4b57bf7a6c603a11bb9,karolinska,4,4+4,3
4,001d865e65ef5d2579c190a0e0350d8f,karolinska,0,0+0,1
...,...,...,...,...,...
10622,ffcd99c47e57ad2934dc6bbf5edf6675,karolinska,0,0+0,2
10624,ffd2841373b39792ab0c84cccd066e31,radboud,0,negative,2
10625,ffdc59cd580a1468eac0e6a32dd1ff2d,radboud,5,4+5,2
10626,ffe06afd66a93258f8fabdef6044e181,radboud,0,negative,3


,image_id,data_provider,isup_grade,gleason_score,split
5,002a4db09dad406c85505a00fb6f6144,karolinska,0,0+0,0
6,003046e27c8ead3e3db155780dc5498e,karolinska,1,3+3,0
10,00412139e6b04d1e1cee8421f38f6e90,karolinska,0,0+0,0
13,004f6b3a66189b4e88b6a01ba19d7d31,karolinska,1,3+3,0
29,00c15b23b30a5ba061358d9641118904,radboud,5,4+5,0
...,...,...,...,...,...
10596,ff339e5fa7be6af83c1b43796092398f,karolinska,4,4+4,0
10603,ff596d5292ab979e9ba7291d0743b3fb,karolinska,0,0+0,0
10618,ffc005d56a21efbd034425623f596984,karolinska,2,3+4,0
10621,ffcbb41626c9267c5c20c4804bd5639a,radboud,4,3+5,0


7465


In [6]:
FOLDED_NUM_TRAIN_IMAGES = train_df.shape[0]
FOLDED_NUM_VALID_IMAGES = valid_df.shape[0]
STEPS_PER_EPOCH = FOLDED_NUM_TRAIN_IMAGES // BATCH_SIZE  # Calculate the steps for keras
VALIDATION_STEPS = FOLDED_NUM_VALID_IMAGES // BATCH_SIZE  # Calculate the same for validation


print('*'*20)
print('Notebook info')
print('Training data : {}'.format(FOLDED_NUM_TRAIN_IMAGES))
print('Validing data : {}'.format(FOLDED_NUM_VALID_IMAGES))
print('Categorical classes : {}'.format(NUM_CLASSES))
print('Training image size : {}'.format(SZ))
print('Training epochs : {}'.format(NUM_EPOCHS))
print('*'*20)

********************
Notebook info
Training data : 7465
Validing data : 2489
Categorical classes : 6
Training image size : 256
Training epochs : 15
********************


In [7]:
coords = np.load('../coordinates/1-16-256-255.npy',allow_pickle=True)

data = TiffFromCoords(coords = coords,
                             df=train_df, 
                             img_dir=IMG_DIR, 
                             batch_size=BATCH_SIZE, 
                             aug_func=None,
                             tf_aug_list = [],
                             one_hot=False)

# do not use augmentation for the val_data, if aug_func is empty (None default), then no augmentation is used.
val_data = TiffFromCoords(coords = coords,
                         df=valid_df, 
                         img_dir=IMG_DIR, 
                         batch_size=BATCH_SIZE, 
                         aug_func=None,
                         one_hot=False)

In [15]:
def qwk_act(x):
    x = K.switch(x>=0, x, 0)
    x = K.switch(x <=5, x, 5)
    return x

In [16]:
lbl_value_counts = train_df['isup_grade'].value_counts()
class_weights = {i: max(lbl_value_counts) / v for i, v in lbl_value_counts.items()}
print('classes weigths:', class_weights)

classes weigths: {0: 1.0, 1: 1.142550911039657, 2: 2.260869565217391, 4: 2.4704519119351103, 3: 2.532066508313539, 5: 2.6031746031746033}


In [17]:
from model.layers import GeneralizedMeanPooling2D

bottleneck = efn.EfficientNetB1( 
    include_top=False, 
    pooling='avg',
    weights='imagenet'
)

#ResNext50, _ = Classifiers.get('resnext50')
#bottleneck = ResNext50(input_shape=(SZ, SZ, 3),
#                       weights='imagenet', include_top=False)


from tensorflow.keras import Sequential
bottleneck = Model(inputs=bottleneck.inputs, outputs=bottleneck.layers[-2].output)
model = Sequential()
model.add(KL.TimeDistributed(bottleneck, input_shape=(NUM_TILES, SZ, SZ, 3)))
model.add(KL.TimeDistributed(KL.BatchNormalization()))
model.add(KL.TimeDistributed(KL.GlobalMaxPooling2D()))
model.add(KL.Flatten())
model.add(KL.BatchNormalization())
model.add(KL.Dropout(.25))
model.add(KL.Dense(512, activation='elu'))
model.add(KL.BatchNormalization())
model.add(KL.Dropout(.25))
model.add(KL.Dense(1, activation=qwk_act, dtype='float32'))

In [18]:
from typing import Optional, List, Union
import datetime

class Config():
    
    num_epochs = 1
    num_grad_accumulates = 5
    step_summary_output = 2
    learning_rate = 1e-3
    model_name = 'ni'
    
config = Config()
    
def accumulated_gradients(gradients: Optional[List[tf.Tensor]],
                          step_gradients: List[Union[tf.Tensor, tf.IndexedSlices]],
                          num_grad_accumulates: int) -> tf.Tensor:
    if gradients is None:
        gradients = [flat_gradients(g) / num_grad_accumulates for g in step_gradients]
    else:
        for i, g in enumerate(step_gradients):
            gradients[i] += flat_gradients(g) / num_grad_accumulates
        
    return gradients

# This is needed for tf.gather like operations.
def flat_gradients(grads_or_idx_slices: tf.Tensor) -> tf.Tensor:
    '''Convert gradients if it's tf.IndexedSlices.
    When computing gradients for operation concerning `tf.gather`, the type of gradients 
    '''
    if type(grads_or_idx_slices) == tf.IndexedSlices:
        return tf.scatter_nd(
            tf.expand_dims(grads_or_idx_slices.indices, 1),
            grads_or_idx_slices.values,
            grads_or_idx_slices.dense_shape
        )
    return grads_or_idx_slices

In [20]:
from tensorflow_addons.metrics import CohenKappa

loss_fn = tf.keras.losses.MeanSquaredError(reduction=tf.keras.losses.Reduction.NONE)


optimizer = tf.keras.optimizers.Adam(config.learning_rate)
train_loss = tf.keras.metrics.Mean('loss/train', dtype=tf.float32)
train_kappa = CohenKappa(num_classes=6, 
                         regression=True, 
                         sparse_labels=True, 
                         weightage='quadratic')
metrics = [train_loss, train_kappa]

current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
train_log_dir = f'logs/{config.model_name}/{current_time}'
train_summary_writer = tf.summary.create_file_writer(train_log_dir)

In [ ]:
def train(config: Config,
          dataset: tf.data.Dataset,
          model: Model):
    global_step = 0
    for e in range(config.num_epochs):
        global_step = train_epoch(config, dataset, model, global_step)
        print(f'{e+1} epoch finished. step: {global_step}')


def train_epoch(config: Config,
                dataset: tf.data.Dataset,
                model: Model,
                start_step: int = 0) -> tf.Tensor:
    '''Train 1 epoch
    '''
    gradients = None
    global_step = start_step
    for i, batch in enumerate(dataset):
        dummy_step = i + start_step * config.num_grad_accumulates
        x_train, y_train = batch
        step_gradients = train_step(x_train, y_train, loss_fn, optimizer)
        gradients = accumulated_gradients(gradients, step_gradients, config.num_grad_accumulates)
        if (dummy_step + 1) % config.num_grad_accumulates == 0:
            gradient_zip = zip(gradients, model.trainable_variables)
            optimizer.apply_gradients(gradient_zip)
            gradients = None
            if (global_step + 1) % config.step_summary_output == 0:
                pass
                #write_train_summary(train_summary_writer, metrics, step=global_step + 1)
            global_step += 1
            print(f"MSE loss {train_loss.result().numpy():.4f} \
                  kappa {train_kappa.result().numpy():.4f} \
                  i {i} gs {global_step}",
                  end="\r")
    train_loss.reset_states()
    train_kappa.reset_states()
    return global_step


@tf.function
def train_step(x_train: tf.Tensor,
               y_train: tf.Tensor,
               loss_fn: tf.keras.losses.Loss,
               optimizer: tf.keras.optimizers.Optimizer):
    '''Train 1 step and return gradients
    '''
    with tf.GradientTape() as tape:
        outputs = model(x_train, training=True)
        loss = tf.reduce_mean(loss_fn(y_train, outputs))
    train_loss(loss)
    train_kappa(y_train, outputs)
    gradients = tape.gradient(loss, model.trainable_variables)
    return gradients


train(config, data(mode='training'), model)

In [20]:
tf.data.experimental.cardinality(data()).numpy()

1493